In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import string
from string import punctuation
import nltk
import re

In [4]:
data = pd.read_csv("C:\\Users\\Dell\\Downloads\\Sentiental Analysis\\Sentiental Analysis\\IMDB Dataset.csv")
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [5]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
stop_words =stopwords.words('english')
def text_processing(text):
	text = text.lower()
	text = re.sub(r'\n',' ',text) #remove new line
	text =re.sub("a-zA-Z0-9$,.","", text)  #remove reference and hashtag
	text = ' '.join([word for word in word_tokenize(text) if word not in stop_words])
	return text

In [7]:
#nltk.download('omw-1.4')
data_copy = data.copy()
data_copy.review = data_copy.review.apply(lambda text: text_processing(text))

In [8]:
le=LabelEncoder()
data_copy['sentiment'] = le.fit_transform(data_copy['sentiment'])
data_copy.tail()

,review,sentiment
49995,thought movie right good job . n't creative or...,1
49996,"bad plot , bad dialogue , bad acting , idiotic...",0
49997,catholic taught parochial elementary schools n...,0
49998,'m going disagree previous comment side maltin...,0
49999,"one expects star trek movies high art , fans e...",0


In [9]:
processed_data = {
    'Sentence':data_copy.review,
    'sentiment':data_copy['sentiment']
}

processed_data = pd.DataFrame(processed_data)
processed_data.head()

,Sentence,sentiment
0,one reviewers mentioned watching 1 oz episode ...,1
1,wonderful little production . < br / > < br / ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically 's family little boy ( jake ) thinks...,0
4,petter mattei 's `` love time money '' visuall...,1


processed_data['sentiment'].value_counts()

In [10]:
corpus = []
for sentence in processed_data['Sentence']:
    corpus.append(sentence)
corpus[0:5]

["one reviewers mentioned watching 1 oz episode 'll hooked . right , exactly happened me. < br / > < br / > first thing struck oz brutality unflinching scenes violence , set right word go . trust , show faint hearted timid . show pulls punches regards drugs , sex violence . hardcore , classic use word. < br / > < br / > called oz nickname given oswald maximum security state penitentary . focuses mainly emerald city , experimental section prison cells glass fronts face inwards , privacy high agenda . em city home many .. aryans , muslims , gangstas , latinos , christians , italians , irish .... scuffles , death stares , dodgy dealings shady agreements never far away. < br / > < br / > would say main appeal show due fact goes shows would n't dare . forget pretty pictures painted mainstream audiences , forget charm , forget romance ... oz n't mess around . first episode ever saw struck nasty surreal , could n't say ready , watched , developed taste oz , got accustomed high levels graphic 

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000)
X = cv.fit_transform(corpus).toarray()
y = processed_data.iloc[:, -1].values
X[0:5]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 2, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [12]:
X.shape

(50000, 5000)

In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
mnb = MultinomialNB()
mnb.fit(X_train , y_train)

MultinomialNB()

In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = mnb.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[6410 1111]
 [1120 6359]]


In [15]:
nb_score = accuracy_score(y_test, y_pred)
print('accuracy',nb_score)

accuracy 0.8512666666666666


In [16]:
import pickle
pickle.dump(cv,open("count-vectorizer.pkl","wb"))
pickle.dump(mnb,open("MovieReview.pkl","wb"))

In [17]:
save_cv = pickle.load(open("count-vectorizer.pkl","rb"))
model = pickle.load(open("MovieReview.pkl","rb"))

In [18]:
def TestModel(sentence):
  sen=save_cv.transform([sentence]).toarray()
  res = model.predict(sen)[0]
  if res==1:
    return "Pos"
  else:
    return "Neg"


In [23]:
sen = "superb"
res = TestModel(sen)
print(res)

Pos
